my ai project about magic telescope. Link to data: https://archive.ics.uci.edu/dataset/159/magic+gamma+telescope , more about data in file located: /data/magic04.names

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
relative_path = '../data/magic04.data'
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv(relative_path, names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
df["class"] = (df["class"] == "g").astype(int)

In [4]:
# for label in cols[:-1]:
#     plt.hist(df[df["class"] == 1][label], color='blue', label='gamma', alpha=0.7, density=True)
#     plt.hist(df[df["class"] == 0][label], color='red', label='hadron', alpha=0.7, density=True)
#     plt.title(label)
#     plt.ylabel("probablitity")
#     plt.xlabel(label)
#     plt.legend()
#     plt.show()

## spliting data across train, valid, test datastes

In [5]:
train, valid, test = np.split(df.sample(frac = 1), [int(0.7 * len(df)), int(0.85 * len(df))])
print(len(train))
print(len(valid))
print(len(test))

13314
2853
2853


/home/kuba/miniconda3/envs/tf/lib/python3.9/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
def scale_data(dataframe, oversample=False):
    x = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
   
    if oversample:
        ros = RandomOverSampler()
        x, y = ros.fit_resample(x, y)
         
    data = np.hstack((x, np.reshape(y, (-1, 1))))
    
    return data, x, y 

In [7]:
train, x_train, y_train = scale_data(train, oversample=True)
valid, x_valid, y_valid = scale_data(valid, oversample=False)
test, x_test, y_test = scale_data(test, oversample=False)

## k-nearest nighbours

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [9]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model = knn_model.fit(x_train, y_train)

In [10]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1032
           1       0.84      0.86      0.85      1821

    accuracy                           0.81      2853
   macro avg       0.79      0.79      0.79      2853
weighted avg       0.81      0.81      0.81      2853



## Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB

In [12]:
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)

In [13]:
y_pred = nb_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.42      0.53      1032
           1       0.73      0.91      0.81      1821

    accuracy                           0.73      2853
   macro avg       0.73      0.66      0.67      2853
weighted avg       0.73      0.73      0.71      2853



## logistic regression

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
log_model = LogisticRegression()
log_model = log_model.fit(x_train, y_train)

In [16]:
y_pred = log_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71      1032
           1       0.84      0.82      0.83      1821

    accuracy                           0.79      2853
   macro avg       0.77      0.77      0.77      2853
weighted avg       0.79      0.79      0.79      2853



## SVM

In [17]:
from sklearn.svm import SVC

In [18]:
svm_model = SVC()
svm_model = svm_model.fit(x_train, y_train)

In [19]:
y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.77      0.78      1032
           1       0.87      0.89      0.88      1821

    accuracy                           0.85      2853
   macro avg       0.84      0.83      0.83      2853
weighted avg       0.85      0.85      0.85      2853



## Neural Network

In [20]:
import tensorflow as tf
print(tf.config.list_physical_devices())

2023-10-07 20:45:52.527043: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-07 20:45:52.807622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 20:45:53.792129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-10-07 20:45:55.055394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-07 20:45:55.223669: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [21]:
# Funkcja do rysowania wykresu straty (loss).
def plot(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.plot(history.history['loss'], label='Strata (Loss)')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_xlabel('Epoka')
    ax1.set_ylabel('Strata')
    ax1.grid(True)
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='Dokładność (Accuracy)')
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax2.set_xlabel('Epoka')
    ax2.set_ylabel('Dokładność')
    ax2.grid(True)
    ax2.legend()

    plt.show()

In [22]:
def train_model(x_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
    nn_model = tf.keras.Sequential([
        tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(num_nodes, activation='relu'),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy',
                metrics=['accuracy'])
    
    with tf.device('/device:GPU:0'):
        history = nn_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        
    return nn_model, history

In [26]:
epochs=100
least_val_loss = float('inf')
least_loss_model = None

for num_nodes in [32, 64]:
    for dropout_prob in [0, 0.2]:
        for lr in [0.001, 0.01]:
            for batch_size in [64, 128]:
                print(f"nodes {num_nodes}, droput {dropout_prob}, lr {lr}, batch size {batch_size}, ")
                model, history = train_model(x_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs)
                # plot(history)
                evaluation = model.evaluate(x_valid, y_valid)
                val_loss = evaluation[0]  
                if val_loss < least_val_loss:
                    least_val_loss = val_loss
                    least_loss_model = model

nodes 32, droput 0, lr 0.001, batch size 64, 
90/90 [==============================] - 0s 783us/step - loss: 0.2918 - accuracy: 0.8787
nodes 32, droput 0, lr 0.001, batch size 128, 
90/90 [==============================] - 0s 1ms/step - loss: 0.2996 - accuracy: 0.8777
nodes 32, droput 0, lr 0.01, batch size 64, 
90/90 [==============================] - 0s 904us/step - loss: 0.3693 - accuracy: 0.8661
nodes 32, droput 0, lr 0.01, batch size 128, 
90/90 [==============================] - 0s 859us/step - loss: 0.3384 - accuracy: 0.8710
nodes 32, droput 0.2, lr 0.001, batch size 64, 
90/90 [==============================] - 0s 868us/step - loss: 0.2903 - accuracy: 0.8798
nodes 32, droput 0.2, lr 0.001, batch size 128, 
90/90 [==============================] - 0s 891us/step - loss: 0.2899 - accuracy: 0.8787
nodes 32, droput 0.2, lr 0.01, batch size 64, 
90/90 [==============================] - 0s 933us/step - loss: 0.2921 - accuracy: 0.8836
nodes 32, droput 0.2, lr 0.01, batch size 128, 
90/

In [27]:
y_pred = least_loss_model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int).reshape(-1)

90/90 [==============================] - 0s 664us/step


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.71      0.79      1032
           1       0.85      0.96      0.90      1821

    accuracy                           0.87      2853
   macro avg       0.88      0.83      0.85      2853
weighted avg       0.87      0.87      0.86      2853

